# 🤖 AI Agent with Reasoning + Tool Use + RAG

This agent uses:
- Local **Llama3 via Ollama**
- **RAG** with ChromaDB
- A **weather tool** the LLM can decide to use
- Chain-of-Thought style prompting


In [ ]:
!pip install -q langchain chromadb requests llama-index

In [ ]:
# Load and embed RAG docs
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os

rag_docs = [
    "In Paris, it's best to carry an umbrella in spring.",
    "Always check the local metro schedule to avoid delays.",
    "Wear comfortable walking shoes when visiting tourist spots in Paris."
]

os.makedirs("rag_docs", exist_ok=True)
for i, text in enumerate(rag_docs):
    with open(f"rag_docs/doc_{i}.txt", "w") as f:
        f.write(text)

all_docs = []
for file in os.listdir("rag_docs"):
    loader = TextLoader(f"rag_docs/{file}")
    all_docs.extend(loader.load())

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = splitter.split_documents(all_docs)

embedding = OllamaEmbeddings(model="llama3")
vectordb = Chroma.from_documents(splits, embedding, persist_directory="./chroma_db")
vectordb.persist()

In [ ]:
# Define the free weather tool
import requests

def get_weather(lat: float, lon: float):
    response = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": lat,
            "longitude": lon,
            "current": "temperature_2m,wind_speed_10m",
            "temperature_unit": "celsius",
            "windspeed_unit": "kmh"
        }
    )
    if response.status_code != 200:
        return {"error": "API failed."}
    return response.json().get("current", {})

In [ ]:
# Reasoning LLM setup
from langchain.llms import Ollama
llm = Ollama(model="llama3")

In [ ]:
# Create reasoning prompt using CoT + tool awareness
question = (
    "You are a travel assistant. A user asks: 'What should I know before going to Paris today?'.\n"
    "You have access to background tips and a weather tool you may use if helpful.\n"
    "If you decide to call the weather tool, determine the correct latitude and longitude for the city.\n"
    "Answer step-by-step with reasoning."
)

response = llm.invoke(question)
print("🤔 LLM Initial Reasoning:")
print(response)

In [ ]:
# Optional: manually detect if LLM wants to call the tool
# Simulate its next step by calling weather tool
lat, lon = 48.8566, 2.3522  # Paris (LLM should have figured this)
weather = get_weather(lat, lon)
weather_text = f"Temperature: {weather.get('temperature_2m', '?')}°C, Wind: {weather.get('wind_speed_10m', '?')} km/h."
print("\n🌦️ Simulated Weather Tool Output:")
print(weather_text)

In [ ]:
# Final query combining tool result + RAG
from langchain.chains import RetrievalQA
retriever = Chroma(persist_directory="./chroma_db", embedding_function=embedding).as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

query = (
    f"The weather in Paris today is: {weather_text}.\n"
    f"Based on this and local travel tips, what should a traveler know?"
)

final_answer = qa.run(query)
print("\n🧠 Final Agent Response:")
print(final_answer)